In [ ]:
import csv
import pandas as pd
import math
import numpy as np
import os
import json
import warnings
warnings.filterwarnings("ignore")

def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def num(v):
    if v == 1:
        return 'one'
    elif v == 2:
        return 'two'
    elif v == 3:
        return 'three'
    elif v == 4:
        return 'four'
    elif v == 5:
        return 'five'
    elif v == 6:
        return 'six'
    elif v == 7:
        return 'seven'
    elif v == 8:
        return 'eight'
    elif v == 9:
        return 'nine'
    

def get_direction_origin(x_a, y_a):
    direction = ""

    if x_a > 0 and y_a > 0:
        direction = "northeast"
    elif x_a > 0 and y_a < 0:
        direction = "southeast"
    elif x_a < 0 and y_a < 0:
        direction = "southwest"
    elif x_a < 0 and y_a > 0:
        direction = "northwest"
    elif x_a > 0:
        direction = "east"
    elif y_a > 0:
        direction = "north"
    elif x_a < 0:
        direction = "west"
    elif y_a < 0:
        direction = "south"

    return direction

def get_direction(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'East'
    elif angle>22.5 and angle<=67.5:
        directions = 'Northeast'
    elif angle>67.5 and angle<=112.5:
        directions = 'North'
    elif angle>112.5 and angle<=157.5:
        directions = 'Northwest'
    elif angle>157.5 and angle<=202.5:
        directions = 'West'
    elif angle>202.5 and angle<=247.5:
        directions = 'Southwest'
    elif angle>247.5 and angle<=292.5:
        directions= 'South'
    elif angle>292.5 and angle<=337.5:
        directions = 'Southeast'

    return directions

def get_direction2(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'West'
    elif angle>22.5 and angle<=67.5:
        directions = 'Southwest'
    elif angle>67.5 and angle<=112.5:
        directions = 'South'
    elif angle>112.5 and angle<=157.5:
        directions = 'Southeast'
    elif angle>157.5 and angle<=202.5:
        directions = 'East'
    elif angle>202.5 and angle<=247.5:
        directions = 'Northeast'
    elif angle>247.5 and angle<=292.5:
        directions= 'North'
    elif angle>292.5 and angle<=337.5:
        directions = 'Northwest'

    return directions

def is_colliding(box1, box2):
    x1_box1 = int(box1['x1'].iloc[0])
    y1_box1 = int(box1['y1'].iloc[0])
    x2_box1 = int(box1['x2'].iloc[0])
    y2_box1 = int(box1['y2'].iloc[0])

    x1_box2 = int(box2['x1'])
    y1_box2 = int(box2['y1'])
    x2_box2 = int(box2['x2'])
    y2_box2 = int(box2['y2'])

    centerx_box1 = int((x1_box1 + x2_box1)/2)
    centery_box1 = int((y1_box1 + y2_box1)/2)

    centerx_box2 = int((x1_box2 + x2_box2)/2)
    centery_box2 = int((y1_box2 + y2_box2)/2)

    center_distance = math.sqrt((centerx_box2 - centerx_box1)**2 + (centery_box2 - centery_box1)**2)

    half_width_sum = ((x2_box1 - x1_box1) + (x2_box2 - x1_box2))/2
    half_height_sum = ((y1_box1 - y2_box1) + (y1_box2 - y2_box2))/2

    #print(center_distance, half_width_sum, half_height_sum)

    if half_width_sum >= center_distance:
        return True

    if half_height_sum >= center_distance:
        return True

    else :
        return False

df = pd.read_csv(f'')
class_order = df[df['frames'] == 'frame0']['class'].tolist()
df = df[df['class'].isin(class_order)].copy()
df['y1'] = -df['y1']
df['y2'] = -df['y2']
df['x1'] -= 256
df['y1'] += 256
df['x2'] -= 256
df['y2'] += 256

con = df['frames'] == 'frame0'
class_id = df.loc[con, 'class']
nob = num(len(list(class_id)))

s1=''
#s1 = f'Initially there are {nob} objects, {num(len(list(class_id))-1)} static and one dynamic object. '
s1 = f'Initially there are {nob} objects, {num(len(list(class_id))-1)} static and one dynamic object. The static objects are '

frame_0 = df[df['frames'] == 'frame0']
frame_0['center'] = frame_0.apply(lambda row: ((row['x1'] + row['x2']) / 2, (row['y1'] + row['y2']) / 2), axis=1)

s2=''
for index, row in frame_0.iterrows():
    current_center = row['center']
    df_without_current = frame_0[frame_0.index != index]  
    current_object = row['class']
    x1_0 = row['x1']
    y1_0 = row['y1']
    d = get_direction_origin(x1_0, y1_0)
    
    nearest_distance = float('inf')
    nearest_object = None
    nearest_center = None
    
    for _, other_row in df_without_current.iterrows():
        other_center = other_row['center']
        distance = calculate_distance(current_center, other_center)
        
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_object = other_row['class']
            nearest_center = other_center
            x1_1 = other_row['x1']
            y1_1 = other_row['y1']

    pos = get_direction2(x1_0, y1_0, x1_1, y1_1)

    #s2 = s2+ f'The {current_object} is located in the {d} direction from the origin and {pos} of {nearest_object}. '
    s2 = s2 + f'{current_object}, '

frame_0 = df[df['frames'] == 'frame0']
class_present = list(frame_0['class'])
changes_det = False
dynamic=[]
for i in range(len(class_present)):
    frame_p = df[df['class'] == class_present[i]]
    frame_p = frame_p.reset_index(drop=True)
    frame_p1 = frame_p.drop(columns=['frames', 'class', 'x2', 'y2'])
    frame_p2 = frame_p.drop(columns=['frames', 'class', 'x1', 'y1'])
    for j in range(1, len(frame_p)):
        prev_frame1 = frame_p1.iloc[j-1]
        curr_frame1 = frame_p1.iloc[j]

        prev_frame2 = frame_p2.iloc[j-1]
        curr_frame2 = frame_p2.iloc[j]

        if not prev_frame1.equals(curr_frame1) and not prev_frame2.equals(curr_frame2):
            changes_det = True
            first_change = j
            break
    
    if changes_det == True:
        dynamic.append((class_present[i], frame_p.iloc[first_change,0], j))

    changes_det = False


sorted_dynamic = sorted(dynamic, key=lambda x:x[2])
dynamic1 = sorted_dynamic[0]
sorted_dynamic = sorted_dynamic[1:]

for i in range(len(sorted_dynamic)):
    frame = sorted_dynamic[i][1]
    ob = sorted_dynamic[i][0]
    frame_n = df[df['frames'] == frame]
    frame_n = frame_n.reset_index(drop=True)
    row = frame_n[frame_n['class'] == ob]
    bbox_1 = row[['x1', 'y1', 'x2', 'y2']]
    ind = bbox_1.index
    ind = ind[0]
    for j in range(len(frame_n)):
        if j!=ind:
            row = frame_n.iloc[j]
            bbox_n = row[['x1', 'y1', 'x2', 'y2']]
            res = is_colliding(bbox_1, bbox_n)
            if res == True:
                cl = list(row[['class']])
                sorted_dynamic[i] = sorted_dynamic[i]+(cl[0],)

frame_dy = df[df['frames'] == dynamic1[1]]
row1 = frame_dy[frame_dy['class'] == dynamic1[0]]
bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
f = dynamic1[2]
frame_dy2 = df[df['frames'] == f'frame{f+3}']
row2 = frame_dy2[frame_dy2['class'] == dynamic1[0]]
bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
x1_0 = bbox1.iloc[0,0]
y1_0 = bbox1.iloc[0,1]
x1_1 = bbox2.iloc[0,0]
y1_1 = bbox2.iloc[0,1]
dy_pos = get_direction_origin(x1_0, y1_0)
dy_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

for i in range(len(sorted_dynamic)):
    if len(sorted_dynamic[i]) >=4:
        collidee = sorted_dynamic[i][0]
        collider = sorted_dynamic[i][3]
        f = sorted_dynamic[i][2]
        #getting the direction of collider
        frames1 = df[df['frames'] == f'frame{f}']
        row1 = frames1[frames1['class'] == collider]
        bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
        frames2 = df[df['frames'] == f'frame{f+1}']
        row2 = frames2[frames2['class'] == collider]
        bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
        x1_0 = bbox1.iloc[0,0]
        y1_0 = bbox1.iloc[0,1]
        x1_1 = bbox2.iloc[0,0]
        y1_1 = bbox2.iloc[0,1]
        collider_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

        #getting the direction of coliidee
        frames1 = df[df['frames'] == f'frame{f}']
        row1 = frames1[frames1['class'] == collidee]
        bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
        frames2 = df[df['frames'] == f'frame{f+1}']
        row2 = frames2[frames2['class'] == collidee]
        bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
        x2_0 = bbox1.iloc[0,0]
        y2_0 = bbox1.iloc[0,1]
        x2_1 = bbox2.iloc[0,0]
        y2_1 = bbox2.iloc[0,1]

        collidee_dir = get_direction(x2_0, y2_0, x2_1, y2_1)
        index=i
        break

s3=''
#s3 = f'The dynamic {dynamic1[0]} is located in the {dy_pos} with respect to the origin at the center. It is moving in the {dy_dir} direction and collides with the {sorted_dynamic[0][0]}. After the collision the {collider} moves in {collider_dir} and {collidee} moves in {collidee_dir} direction. '
#s3 = f'The dynamic {dynamic1[0]} collides with the {sorted_dynamic[0][0]}. '
s3 = f'The dynamic {dynamic1[0]} collides with the {sorted_dynamic[0][0]}.It is moving in the {dy_dir} direction and collides with the {sorted_dynamic[0][0]}. After the collision the {collider} moves in {collider_dir} and {collidee} moves in {collidee_dir} direction.  '
s4=''
for i in range(index+1, len(sorted_dynamic)):
    if len(sorted_dynamic[i]) >=4:
        collidee = sorted_dynamic[i][0]
        collider = sorted_dynamic[i][3]
        f = sorted_dynamic[i][2]

        #getting the direction of collider
        frames1 = df[df['frames'] == f'frame{f}']
        row1 = frames1[frames1['class'] == collider]
        bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
        frames2 = df[df['frames'] == f'frame{f+1}']
        row2 = frames2[frames2['class'] == collider]
        bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
        x1_0 = bbox1.iloc[0,0]
        y1_0 = bbox1.iloc[0,1]
        x1_1 = bbox2.iloc[0,0]
        y1_1 = bbox2.iloc[0,1]
        collider_dir = get_direction(x1_0, y1_0, x1_1, y1_1)

        #getting the direction of coliidee
        frames1 = df[df['frames'] == f'frame{f}']
        row1 = frames1[frames1['class'] == collidee]
        bbox1 = row1[['x1', 'y1', 'x2', 'y2']]
        frames2 = df[df['frames'] == f'frame{f+1}']
        row2 = frames2[frames2['class'] == collidee]
        bbox2 = row2[['x1', 'y1', 'x2', 'y2']]
        x1_0 = bbox1.iloc[0,0]
        y1_0 = bbox1.iloc[0,1]
        x1_1 = bbox2.iloc[0,0]
        y1_1 = bbox2.iloc[0,1]
        collidee_dir = get_direction(x1_0, y1_0, x1_1, y1_1)


        s4 = s4+f'The {collider} collides with the {collidee}. The {collider} moves in the {collider_dir} and the {collidee} moves in the {collidee_dir} direction. '
        #s4 = s4 + f'The {collider} collides with the {collidee}. '

s=s1+s2+s3+s4

3:
Initially there are five objects, four static and one dynamic object. The static objects are teal aluminium cube, olive cardboard sphere, purple cardboard cube, purple aluminium cube, teal aluminium sphere, The dynamic purple aluminium cube collides with the purple cardboard cube.It is moving in the Northwest direction and collides with the purple cardboard cube. After the collision the purple aluminium cube moves in Northwest and purple cardboard cube moves in West direction.  The purple aluminium cube collides with the teal aluminium cube. The purple aluminium cube moves in the Northwest and the teal aluminium cube moves in the Northwest direction. The teal aluminium cube collides with the olive cardboard sphere. The teal aluminium cube moves in the West and the olive cardboard sphere moves in the West direction. ",

24: "Initially there are five objects, four static and one dynamic object. The static objects are teal aluminium cube, olive cardboard sphere, purple cardboard sphere, purple aluminium sphere, teal aluminium sphere, The dynamic teal aluminium cube collides with the purple cardboard sphere.It is moving in the Northeast direction and collides with the purple cardboard sphere. After the collision the olive cardboard sphere moves in  and teal aluminium sphere moves in  direction.  The teal aluminium cube collides with the olive cardboard sphere. The teal aluminium cube moves in the North and the olive cardboard sphere moves in the  direction. ",

Initially there are five objects, four static and one dynamic object. The teal aluminium cube is located in the southwest direction from the origin and East of olive cardboard sphere. The olive cardboard sphere is located in the northwest direction from the origin and  of teal aluminium cube. The purple cardboard cube is located in the south direction from the origin and South of teal aluminium cube. The purple aluminium cube is located in the southeast direction from the origin and Southeast of purple cardboard cube. The teal aluminium sphere is located in the northwest direction from the origin and North of teal aluminium cube. The dynamic purple aluminium cube is located in the southeast with respect to the origin at the center. It is moving in the Northwest direction and collides with the purple cardboard cube. After the collision the purple aluminium cube moves in Northwest and purple cardboard cube moves in West direction. The purple aluminium cube collides with the teal aluminium cube. The purple aluminium cube moves in the Northwest and the teal aluminium cube moves in the West. The teal aluminium cube collides with the olive cardboard sphere. The teal aluminium cube moves in the Northwest and the olive cardboard sphere moves in the Northwest. ",